# About objects and classes

Modern C++ does not revolutionize the object-oriented features but rather brings many small improvements for most of the identified flaws.

## Improvement of object constructions

#### Delegating constructors

In [1]:
#include <iostream>

In [3]:
struct ClassA {
    ClassA() : ClassA(0) { m_default = true ; }
    ClassA( int a_x ) : m_x {a_x} { m_default = false ; }
    int m_x ;
    bool m_default ;
} ;

In [4]:
void display( ClassA const & a_obj )
{
  if (a_obj.m_default) { std::cout<<"(default)"<<a_obj.m_x<<std::endl ; }
  else { std::cout<<"(explicit)"<<a_obj.m_x<<std::endl ; }
}

In [5]:
ClassA a1 {}, a2 {0} ;
display(a1) ;
display(a2) ;

(default)0
(explicit)0


#### Initialization of data members

In [6]:
struct ClassB {
  ClassB() {} ;
  ClassB( int a_x ) : m_x {a_x} {}
  int m_x = 0 ;
} ;

In [7]:
ClassB b0 {} ;
ClassB b1 {1} ;
std::cout << b0.m_x << std::endl ;
std::cout << b1.m_x << std::endl ;

0
1


#### Inheritance and constructors

In [9]:
#include <iostream>

In [10]:
struct ClassA {
    ClassA( int ) { std::cout << "ClassA(int)" << std::endl ; }
    ClassA( int, int ) { std::cout << "ClassA(int,int)" << std::endl ; }
} ;

In [11]:
struct ClassB : public ClassA {
  using ClassA::ClassA ; // either A::A( int) or A::A( int, int ) can be used
                         // as if they were declared as B::B( int) and B::B( int, int )
} ;

In [12]:
ClassB b1 {1} ;
ClassB b2 {1,2} ;

ClassA(int)
ClassA(int,int)


## Improvement of declaration of member functions

#### Forbid a function

In [16]:
// class cannot be copied in C++03
class no_copies {
  public:
    no_copies() {}
  private:
    no_copies( const no_copies & ) ;
    no_copies & operator=( const no_copies & ) ;
} ;

In [17]:
// class cannot be copied in C++11
class no_copies_v2 {
  public:
    no_copies_v2() {}
    no_copies_v2( no_copies_v2 const & ) = delete ;
    no_copies_v2 & operator=( no_copies_v2 const & ) = delete ;
} ;

#### We can now prevent certain implicit conversions

In [18]:
struct FooStruct {
  void foo_method(short) {}
  void foo_method(int) = delete ;
} ;

In [19]:
FooStruct s ;
s.foo_method(42) ; // Error, int overload declared deleted
s.foo_method(static_cast<short>(42)) ; // OK

input_line_26:3:3: error: call to deleted member function 'foo_method'
s.foo_method(42) ; // Error, int overload declared deleted
~~^~~~~~~~~~
input_line_25:3:8: note: candidate function has been explicitly deleted
  void foo_method(int) = delete ;
       ^
input_line_25:2:8: note: candidate function
  void foo_method(short) {}
       ^


Interpreter Error: 


#### Modify the signature but keep the default implementation

In [1]:
class Y {
  public:
    Y & operator=( Y const & ) = default ; // Make it explicit
    virtual ~Y() = default ; // Add virtual
  protected:
    Y() = default ; // Change access
} ; 

#### Control the redefinition of virtual methods

In [21]:
class ClassA
 {
  public :
    virtual void fct1() =0 ;
    virtual void fct2( int ) =0 ;
    virtual void fct3( bool ) =0 ;
 } ;

In [22]:
class ClassB : public ClassA
 {
  public :
    void fct1() override ;       // OK
    void ft2( int ) override ;   // error: A::ft2 does not exist
    void fct2( bool ) override ; // error: not the good types
 } ;

input_line_29:5:22: error: only virtual member functions can be marked 'override'
    void ft2 ( int ) override ;   // error: A::ft2 does not exist
                     ^~~~~~~~~
input_line_29:6:24: error: non-virtual member function marked 'override' hides virtual member function
    void fct2 ( bool ) override ; // error: not the good types
                       ^
input_line_28:5:18: note: hidden overloaded virtual function '__cling_N520::A::fct2' declared here: type mismatch at 1st parameter ('int' vs 'bool')
    virtual void fct2( int ) =0 ;
                 ^


Interpreter Error: 

#### Forbid redefining virtual methods and abstract classes

In [ ]:
struct ClassA {
    virtual void fct1() =0 ;
    virtual void fct2( int ) =0 ;
} ;

In [ ]:
struct ClassB : public A {
    virtual void fct3( bool ) final ;
} ;

In [ ]:
struct ClassC final : public B {
    void fct1() override ;       // OK
    void fct3( bool ) override ; // error: B::fct3 is final
} ;

In [ ]:
struct D : public C {} ;               // error: C is final         

## Hiding is still an issue

When one use both inheritance and overloading, i.e. multiple functions with the same name but different signatures (the number or type of arguments), how does it work?

In [5]:
struct Base
 {
  void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
  void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
 } ;

In [6]:
struct Derived : public Base
 {
  void display( float ) { std::cout<<"Derived::display( float )"<<std::endl ; }
  void display( double ) { std::cout<<"Derived::display( double )"<<std::endl ; }
 } ;

In [7]:
void check( Derived obj )
 { 
  obj.display(42) ;
  obj.display(3.14f) ;
  obj.display(3.14) ;
 }

input_line_13:3:7: error: call to member function 'display' is ambiguous
  obj.display(42) ;
  ~~~~^~~~~~~
input_line_12:3:8: note: candidate function
  void display( float ) { std::cout<<"Derived::display( float "<<std::endl ; }
       ^
input_line_12:4:8: note: candidate function
  void display( double ) { std::cout<<"Derived::display( double )"<<std::endl ; }
       ^


Interpreter Error: 

When compiling `obj.mf(...)`, the compiler follow those steps:
1. search some member function named `mf` in the class of `obj`;
2. while not found at least one `mf`, move to its base classes, one after the other;
3. in the selected class, within all overloaded `mf` functions, select the one which fit better the call arguments `(...)`.

### Good old-fashioned practice

When you redefine an inherited member function, **redefine all the base class functions which share the same name**, or you will hide some of them.

## Reminders about virtual member functions

Let's make all the member functions virtual, and modify `check()` so to take as input a reference to the base class, so that we can provide any derived class instance.

In [8]:
#include<iostream>

In [9]:
struct Base
 {
  virtual void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
  virtual void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
 } ;

In [10]:
struct Derived : public Base
 {
  virtual void display( float ) { std::cout<<"Derived::display( float "<<std::endl ; }
  virtual void display( double ) { std::cout<<"Derived::display( double )"<<std::endl ; }
 } ;

In [11]:
void check( Base * obj )
 {
  obj->display(42) ;
  obj->display(3.14f) ;
  obj->display(3.14) ;
 }

input_line_17:5:8: error: call to member function 'display' is ambiguous
  obj->display(3.14) ;
  ~~~~~^~~~~~~
input_line_15:3:16: note: candidate function
  virtual void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
               ^
input_line_15:4:16: note: candidate function
  virtual void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
               ^


Interpreter Error: 

The ambiguous line is now `obj.display(3.14)`: the type of `*obj` is `Base`, and the compiler hesitate to convert its `double` argument into either an `int` or a `float`.

### Good old-fashioned practice

When you define a base class with virtual functions, overload those functions for all the expected type of arguments that you expect in the derived classes.

In [12]:
struct Base
 {
  virtual void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
  virtual void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
  virtual void display( double ) { std::cout<<"Base::display( double )"<<std::endl ; }
 } ;

In [ ]:
struct Base
 {
  virtual void display( int ) { std::cout<<"Base::display( int )"<<std::endl ; }
  virtual void display( float ) { std::cout<<"Base::display( float )"<<std::endl ; }
  virtual void display( double ) { std::cout<<"Base::display( double )"<<std::endl ; }
  virtual ~Base() {}
 } ;

1. **the size of all objects**, for this class and any derived class, **is increased by the size of a pointer** (pointing to the *virtual table* of the object real class).

### The recommended use of virtual functions

Because of the cost, virtual functions are rather used for large and not numerous objects: the upper software layers of your application. For what concerns the numerous and small objects, it is preferable to use templates (which slow down the compilation but have no effect during execution).

# Take away

- The new `delete` keyword forbids more explicitly some functions.
- The new `override` keyword helps to detect typos when redefining inherited member functions.
- Still, be aware of the hiding pitfall if you overload with different signatures.
- Avoid the costly virtual functions for low level objects.

# Questions ?

# Exercise

In the code below:
- insert one `= delete`, one `= default` and one `override` ;
- just in case, give default values to the members variables ;
- prevent the compiler from implicitly transforming a double into a particle.
- what is still lacking?

In [24]:
%%file tmp.objets.cpp

#include <cstdlib> // pour std::rand()
#include <iostream>
#include <string>

class Particle
 {
  public  :
    Particle( double a_mass ) : m_mass {a_mass} {}
    double mass() { return m_mass ; }
    virtual std::string name() { return "Particle" ; }
    ~Particle() {}
  private  :
    Particle( Particle const & ) ; // non copiable
    double m_mass ;
 } ;

class ChargedParticle : public Particle
 {
  public  :
    ChargedParticle( double a_mass, double a_charge )
     : Particle(a_mass), m_charge {a_charge} {}
    double charge() { return m_charge ; }
    virtual std::string name() { return "ChargedParticle" ; }
  private  :
    double m_charge ;
 } ;

void print( Particle & a_p  )
 {
  std::cout << a_p.name() << std::endl ;          
  std::cout << "  mass = " << p.mass() << std::endl ;         
 }

int main()
 {
  for ( int i = 0 ; i < 5 ; ++i )
   {
    if ( std::rand() < (.5*RAND_MAX) )
     {
      Particle p {2} ;
      print(p) ;
     }
    else
     {
      ChargedParticle p {1,1} ;
      print(p) ;
      std::cout << "  charge = " << p.charge() << std::endl ;         
     }
   }
 }

Overwriting tmp.objets.cpp


In [25]:
!rm -f tmp.objets.exe && g++ -std=c++17 tmp.objets.cpp -o tmp.objets.exe

In [26]:
!./tmp.objets.exe

ChargedParticle
  mass = 1
  charge = 1
Particle
  mass = 2
ChargedParticle
  mass = 1
  charge = 1
ChargedParticle
  mass = 1
  charge = 1
ChargedParticle
  mass = 1
  charge = 1


© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*